In [1]:
import csv
def read_csv(filename):
    filename += ".csv"
    filename = "CSV/" + filename 
    lists = []
    with open(filename, encoding="UTF-8") as f:
        for list in csv.reader(f):
            list[1] = float(list[1])
            list[2] = float(list[2])
            lists.append(list)
    return lists

In [2]:
lists_IKEA = read_csv("IKEA")
lists_COSTCO = read_csv("COSTCO")
lists_LALAPORT = read_csv("LALAPORT")
lists_NITORI = read_csv("NITORI")
lists_DECOHOME = read_csv("DECOHOME")
lists_station = read_csv("station")
lists_all_stations = read_csv("all_stations")
lists_IC = read_csv("IC")

In [3]:
# import the module to calculate the distance (km) of two places
# from latitude and longitude
from geopy.distance import geodesic

# define the function 
# input: list of shops and list of traffic or competitors
# output: None
# append the nearest distance into the list
def dist(lists_shop, lists_traffic):
    infty = 100000000000000000000000
    for i in range(len(lists_shop)):
        place_shop = (lists_shop[i][1], lists_shop[i][2])
        mini_dist = infty

        for j in range(len(lists_traffic)):
            place_traffic = (lists_traffic[j][1], lists_traffic[j][2])
            new_dist = geodesic(place_shop, place_traffic).km
            mini_dist = min(mini_dist, new_dist)

        lists_shop[i].append(mini_dist)

In [4]:
# calculate the distance to the nearest hub station
# calculate the distance to the nearest Inter Change
dist(lists_IKEA, lists_station)
dist(lists_IKEA, lists_IC)
dist(lists_COSTCO, lists_station)
dist(lists_COSTCO, lists_IC)

# calculate the distance to the nearest LALAPORT (competitor)
dist(lists_IKEA, lists_LALAPORT)
dist(lists_COSTCO, lists_LALAPORT)

# calculate the distance to the nearest NITORI (competitor)
dist(lists_IKEA, lists_NITORI)
dist(lists_COSTCO, lists_NITORI)

# calculate the distance to the nearest DECOHOME (competitor)
dist(lists_IKEA, lists_DECOHOME)
dist(lists_COSTCO, lists_DECOHOME)

# calculate the distance to the nearest all stations
dist(lists_IKEA, lists_all_stations)
dist(lists_COSTCO, lists_all_stations)

In [5]:
# define the functoin to calculate the average distance from the subject (IKEA and COSTCO) to the nearest place (station, iC, LALAPORT, and NITORI)
def calculate_average(column_number, place):
    print("[{}]".format(place))
    
    # calculate the average distance from IKEA to the nearest place
    sum_IKEA = 0
    for i in range(len(lists_IKEA)):
        sum_IKEA += lists_IKEA[i][column_number]

    avg_IKEA = sum_IKEA / len(lists_IKEA)
    print("the average distance from IKEA to the nearest {}: {}km".format(place, round(avg_IKEA, 1)))

    # calculate the average distance from COSTCO to the nearest place
    sum_COSTCO = 0
    for i in range(len(lists_COSTCO)):
        sum_COSTCO += lists_COSTCO[i][column_number]

    avg_COSTCO= sum_COSTCO / len(lists_COSTCO) 
    print("the average distance from COSTCO to the nearest {}: {}km".format(place, round(avg_COSTCO, 1)))

    if(avg_IKEA < avg_COSTCO ):
        print("IKEA is closer to the {} in terms of distance than COSTCO\n".format(place))
    else:
        print("COSTCO is closer to the {} in terms of distance than IKEA\n".format(place))

In [6]:
calculate_average(3, "hub station")
calculate_average(4, "Inter Change")
calculate_average(5, "LALAPORT (competitor)")
calculate_average(6, "NITORI (competitor)")
calculate_average(7, "DECOHOME (competitor)")
calculate_average(8, "all stations")

[hub station]
the average distance from IKEA to the nearest hub station: 13.6km
the average distance from COSTCO to the nearest hub station: 39.5km
IKEA is closer to the hub station in terms of distance than COSTCO

[Inter Change]
the average distance from IKEA to the nearest Inter Change: 36.3km
the average distance from COSTCO to the nearest Inter Change: 32.8km
COSTCO is closer to the Inter Change in terms of distance than IKEA

[LALAPORT (competitor)]
the average distance from IKEA to the nearest LALAPORT (competitor): 27.0km
the average distance from COSTCO to the nearest LALAPORT (competitor): 35.2km
IKEA is closer to the LALAPORT (competitor) in terms of distance than COSTCO

[NITORI (competitor)]
the average distance from IKEA to the nearest NITORI (competitor): 18.1km
the average distance from COSTCO to the nearest NITORI (competitor): 28.3km
IKEA is closer to the NITORI (competitor) in terms of distance than COSTCO

[DECOHOME (competitor)]
the average distance from IKEA to th

In [7]:
def make_SQL(lists_table, SQL_table_name, SQL_table_id):
    # create the SQL command for inserting the data
    for i in range(len(lists_table)):
        sql_command = "INSERT INTO `{}` (`{}`, `name`, `latitude`, `longitude`) VALUES ('{}', '{}', '{}', '{}');".format(SQL_table_name, SQL_table_id, i+1, lists_table[i][0], lists_table[i][1], lists_table[i][2])
        with open("SQL/INSERT_PLACE.sql", "a", encoding="UTF-8") as f:
            f.write(sql_command + "\n")

In [8]:
# create the SQL command for inserting the IKEA data
make_SQL(lists_IKEA, "ikeas", "ikea_id")

# create the SQL command for inserting the COSTCO data
make_SQL(lists_COSTCO, "costcos", "costco_id")

# create the SQL command for inserting the station data
make_SQL(lists_station, "stations", "station_id")

# create the SQL command for inserting the Inter Chnage data
make_SQL(lists_IC, "ics", "ic_id")

# create the SQL command for inserting the LALAPORT data
make_SQL(lists_LALAPORT, "lalaports", "lalaport_id")

# create the SQL command for inserting the NITORI data
make_SQL(lists_NITORI, "nitoris", "nitori_id")

# create the SQL command for inserting the DECOHOME data
make_SQL(lists_DECOHOME, "decohomes", "decohome_id")

# create the SQL command for inserting the all station data
make_SQL(lists_all_stations, "all_stations", "all_station_id")

In [9]:
# define function to craete the data for relatinoal database
from geopy.distance import geodesic
def make_relation(table_name, lists_shop, lists_traffic_competitor, number_traffic_competitor):
    for i in range(len(lists_shop)):
        shop_lat_long = (lists_shop[i][1], lists_shop[i][2])
        
        for j in range(len(lists_traffic_competitor)):
            traffic_competitor_lat_long = (lists_traffic_competitor[j][1], lists_traffic_competitor[j][2])
            dist = geodesic(shop_lat_long, traffic_competitor_lat_long).km
            if (lists_shop[i][number_traffic_competitor] == dist):
                relation_list = [i+1, j+1, dist]
                table_name.append(relation_list)
                break

In [10]:
# apply the function and create the eight lists for the relational database
relation_ikea_station = []
make_relation(relation_ikea_station, lists_IKEA, lists_station, 3)

relation_ikea_ic = []
make_relation(relation_ikea_ic, lists_IKEA, lists_IC, 4)

relation_ikea_lalaport = []
make_relation(relation_ikea_lalaport, lists_IKEA, lists_LALAPORT, 5)

relation_ikea_nitori = []
make_relation(relation_ikea_nitori, lists_IKEA, lists_NITORI, 6)

relation_ikea_decohome = []
make_relation(relation_ikea_decohome, lists_IKEA, lists_DECOHOME, 7)

relation_ikea_all_station = []
make_relation(relation_ikea_all_station, lists_IKEA, lists_all_stations, 8)


relation_costco_station = []
make_relation(relation_costco_station, lists_COSTCO, lists_station, 3)

relation_costco_ic = []
make_relation(relation_costco_ic, lists_COSTCO, lists_IC, 4)

relation_costco_lalaport = []
make_relation(relation_costco_lalaport, lists_COSTCO, lists_LALAPORT, 5)

relation_costco_nitori = []
make_relation(relation_costco_nitori, lists_COSTCO, lists_NITORI, 6)

relation_costco_decohome = []
make_relation(relation_costco_decohome, lists_COSTCO, lists_DECOHOME, 7)

relation_costco_all_station = []
make_relation(relation_costco_all_station, lists_COSTCO, lists_all_stations, 8)

In [11]:
# define function to make SQL code to create the relational tables 
# each table contains 
# column1. the id of shops (IKEA or COSTCO)
# column2. the id of the nearest traffic places (station or IC) or competitors (LALAPORT or NITORI)
# column3. the distcande bewtween column1 and column2
def create_SQL_relational(shop_name, traffic_competitor_name):
   sql_command = "CREATE TABLE `relation_{}_{}` ( `{}_id` INT NOT NULL, `{}_id` INT NOT NULL, `dist_{}` DOUBLE NOT NULL, PRIMARY KEY (`{}_id`, `{}_id`)) ENGINE = InnoDB;".format(shop_name, traffic_competitor_name, shop_name, traffic_competitor_name, traffic_competitor_name, shop_name, traffic_competitor_name)
   with open("SQL/CREATE_RELATIONAL.sql", "a", encoding="UTF-8") as f:
      f.write(sql_command + "\n")

In [12]:
# generate the code to create the eight tables
create_SQL_relational("ikea", "station")
create_SQL_relational("ikea", "ic")
create_SQL_relational("ikea", "lalaport")
create_SQL_relational("ikea", "nitori")
create_SQL_relational("ikea", "decohome")
create_SQL_relational("ikea", "all_station")

create_SQL_relational("costco", "station")
create_SQL_relational("costco", "ic")
create_SQL_relational("costco", "lalaport")
create_SQL_relational("costco", "nitori")
create_SQL_relational("costco", "decohome")
create_SQL_relational("costco", "all_station")

In [13]:
# define function to make SQL code to create the foreign keys
def alter_SQL_relational(shop_name, traffic_competitor_name):
   sql_command1 = "ALTER TABLE `relation_{}_{}` ADD FOREIGN KEY (`{}_id`) REFERENCES `{}s`(`{}_id`) ON DELETE RESTRICT ON UPDATE RESTRICT;".format(shop_name, traffic_competitor_name, shop_name, shop_name, shop_name)
   sql_command2 = "ALTER TABLE `relation_{}_{}` ADD FOREIGN KEY (`{}_id`) REFERENCES `{}s`(`{}_id`) ON DELETE RESTRICT ON UPDATE RESTRICT;".format(shop_name, traffic_competitor_name, traffic_competitor_name, traffic_competitor_name, traffic_competitor_name)
   with open("SQL/ALTER_RELATIONAL.sql", "a", encoding="UTF-8") as f:
      f.write(sql_command1 + "\n" + sql_command2 + "\n")

In [14]:
# generate the ALTER code
alter_SQL_relational("ikea", "station")
alter_SQL_relational("ikea", "ic")
alter_SQL_relational("ikea", "lalaport")
alter_SQL_relational("ikea", "nitori")
alter_SQL_relational("ikea", "decohome")
alter_SQL_relational("ikea", "all_station")

alter_SQL_relational("costco", "station")
alter_SQL_relational("costco", "ic")
alter_SQL_relational("costco", "lalaport")
alter_SQL_relational("costco", "nitori")
alter_SQL_relational("costco", "decohome")
alter_SQL_relational("costco", "all_station")

In [15]:
# define the function to insert the data into relational table 
def insert_SQL_relational(list_table, shop_name, traffic_competitor_name):
   for i in range(len(list_table)):
      sql_command = "INSERT INTO `relation_{}_{}` (`{}_id`, `{}_id`, `dist_{}`) VALUES ('{}', '{}', '{}');".format(shop_name, traffic_competitor_name, shop_name, traffic_competitor_name, traffic_competitor_name, list_table[i][0], list_table[i][1], list_table[i][2])
      with open("SQL/INSERT_RELATIONAL.sql", "a", encoding="UTF-8") as f:
         f.write(sql_command + "\n")

In [16]:
# generate the SQL codes
insert_SQL_relational(relation_ikea_station, "ikea", "station")
insert_SQL_relational(relation_ikea_ic, "ikea", "ic")
insert_SQL_relational(relation_ikea_lalaport, "ikea", "lalaport")
insert_SQL_relational(relation_ikea_nitori, "ikea", "nitori")
insert_SQL_relational(relation_ikea_decohome, "ikea", "decohome")
insert_SQL_relational(relation_ikea_all_station, "ikea", "all_station")

insert_SQL_relational(relation_costco_station, "costco", "station")
insert_SQL_relational(relation_costco_ic, "costco", "ic")
insert_SQL_relational(relation_costco_lalaport, "costco", "lalaport")
insert_SQL_relational(relation_costco_nitori, "costco", "nitori")
insert_SQL_relational(relation_costco_decohome, "costco", "decohome")
insert_SQL_relational(relation_costco_all_station, "costco", "all_station")